# LLM Prompt Engineering Lab
## Tree of Thought Prompting with GPT4o-mini

Preliminaries: import libraries, read in the OpenAI API key and pass it to an OpenAI object constructor

In [ ]:
# Import libraries, set API key
!pip install anytree
from anytree import Node, RenderTree

import sys
sys.path.append('/content/sample_data')
import ASCIImaze
import os
import re

# This chunk of code reads an OpenAI API key from a chosen environment variable, which must be predefined.
# For my own dev, I'm using my personal API key. This will not be provided to others, you must use one you have
# access to.
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

from openai import OpenAI

client = OpenAI(
    api_key="Insert key here")


<b>First prompt:</b> Recognize the ASCII maze, and format it as a single-line string, with the different lines of the maze delimited by "|"

In [53]:
extract_maze_system_prompt = " Recognize the ASCII maze, and format it as a single-line string, with the different lines of the maze delimited by '|'. Return nothing but the string representing the maze."

<b>Second prompt:</b> Format the 3x3 neighborhood in the same manner as the ASCII maze.

In [54]:
generate_thought_context_system_prompt = """
You are analyzing a 3x3 neighborhood of the maze.
Convert it into a single-line string where each row is separated by the character '|'.
Return only the single-line string, with no extra text or formatting. 11 characters total including the delimiter

The output MUST follow this regex: ^.{3}\|.{3}\|.{3}$. Make sure to retain spaces.

Examples (showing correct format):

Input:
###
 *
 #

Output:
###| * | #

Input:
# #
#*#
#E#

Output:
# #|#*#|#E#

Input:
#S#
#*
# #

Output:
#S#|#* |# #

"""

<>:6: SyntaxWarning: invalid escape sequence '\|'
<>:6: SyntaxWarning: invalid escape sequence '\|'
/tmp/ipython-input-1658719250.py:6: SyntaxWarning: invalid escape sequence '\|'
  The output MUST follow this regex: ^.{3}\|.{3}\|.{3}$. Make sure to retain spaces.


<b>Third prompt:</b> Based on the refomatted ASCII maze and neighborhood from prompts 1 and 2 above, choose the next move to make.

In [55]:

generate_thought_system_prompt = """
You are a maze navigation agent making a single move at a time.
You must choose exactly one move from the list of valid options.
Never invent or imagine directions that are not in the list.
Never describe or explain your reasoning.
Only output one capital letter on a single line, with no punctuation or commentary.

Valid moves: {vmove}

Your entire output must be exactly one of these characters — nothing else.
Example:
If valid moves are "L, D", valid output: D
If valid moves are "R", valid output: R

"""

Output rule: reply with **one uppercase letter** (U/D/L/R) only — no words, punctuation, or reasoning.

In [56]:
# Possible models:
# gpt-3.5-turbo
# gpt-4o-mini
total_input_tokens = 0
total_output_tokens = 0
# get_completion is a helper method suggested in the DeepLearning.ai course on prompt engineering.
# I modified this version to use the OpenAI 1.0 implementation. The "client" object was passed an API key in line 24
def get_completion(system_prompt, user_prompt, my_model="gpt-4o-mini"):
    global total_input_tokens, total_output_tokens
    completion = client.chat.completions.create(
        model=my_model,
        messages=[
            {"role": "system",
             "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    usage = completion.usage
    total_input_tokens += usage.prompt_tokens
    total_output_tokens += usage.completion_tokens


    return completion.choices[0].message


# -------------------------------------------------------------------------------------------------------------

# Utility function for displaying trees. Core code from the anytree package documentation.
def render_that_tree(mytree):
    for pre, _, node in RenderTree(mytree):
        print("%s%s" % (pre, node.name))

# ---------------------------------------------------------------------------------------------------------------

# Backtracking subroutine for Tree of Thought. Updates teh parent's state info to prune the dead-end branch we are
#  backtracking from (by setting that position in the stored 3x3 to '#').

def tree_backtrack(deadend_node, curr_move):

    curr_node = deadend_node.parent

    if deadend_node.move == 'D':
        temp = list(curr_node.name)
        temp[9] = '#'
        curr_node.name = ''.join(temp)

    elif deadend_node.move == 'U':
        temp = list(curr_node.name)
        temp[1] = '#'
        curr_node.name = ''.join(temp)

    elif deadend_node.move == 'L':
        temp = list(curr_node.name)
        temp[4] = '#'
        curr_node.name = ''.join(temp)

    elif deadend_node.move == 'R':
        temp = list(curr_node.name)
        temp[6] = '#'
        curr_node.name = ''.join(temp)

    else:
        print("Illegal move in backtracking")
        return None

    return curr_node

# ----------------------------------------------------------------------------------------------------------------

# Add node subroutine for Tree of Thought. Pretty short, but putting it here makes teh main loop cleaner.
#  This also pushes the tree updates out to subroutines, which is common and familiar. Leaves just teh LLM and game
#  logic in the main loop, which I think is good.
def add_node(curr_node, LLM_move_cleaned):
    # New node is a child of the current node. State has to be determined on next iteration. The move to reach this
    #  node from its parent is recorded as the "move" attribute. The move to reach its parent from this node
    #  is recorded as the "backtrack" attribute.

    # Have to determine the backtrack attribute by reversing LLM_cleaned_move. Another elif tree here.
    if LLM_move_cleaned.find('D') >= 0:
        rev = 'U'

    elif LLM_move_cleaned.find('U') >= 0:
        rev = 'D'

    elif LLM_move_cleaned.find('L') >= 0:
        rev = 'R'

    elif LLM_move_cleaned.find('R') >= 0:
        rev = 'L'

    else:
        print("Not a legal move")
        return None

    temp = Node(parent=curr_node, name="", move=LLM_move_cleaned, backtrack=rev)

    return temp


# ----------------------------------------------------------------------------------------------------------

def evaluate_thought(curr_local, curr_node):
    # Pre: curr_local is the 3x3 neighborhood for the current location in the maze.
    # Format ###|#*#|###
    # curr_node is the current node of the tree, the center point of the 3x3 local neighborhood.
    # Post: none
    # Returns: List of valid moves, each a 1-character string from {U, D, L, R}

    # Create output list, convert input string to a list.

    # Debug
    print(f"evaluate thought recevied {curr_local}")
    # Debug

    valid_moves = []
    maze_neigh = curr_local
    # Checking validity of each possible move
    if maze_neigh[1] in [' ', 'S', 'E']:
        valid_moves.append('U')
    if maze_neigh[4] in [' ', 'S', 'E']:
        valid_moves.append('L')
    if maze_neigh[6] in [' ', 'S', 'E']:
        valid_moves.append('R')
    if maze_neigh[9] in [' ', 'S', 'E']:
        valid_moves.append('D')

    # If there’s more than one open move, drop the backtrack one.
    if len(valid_moves) > 1 and curr_node.backtrack in valid_moves:
        valid_moves.remove(curr_node.backtrack)

    # If no moves left, the only option is to backtrack.
    if not valid_moves:
        valid_moves.append(curr_node.backtrack)

    return valid_moves

# ----------------------------------------------------------------------------------------------------------

# Guards for 1st iteration, context request.
ready_for_input = False
need_context_update = True
mazeTreeStart = Node(name="", move="", backtrack="")
curr_node = mazeTreeStart
# Debug
counter = 0

print(ASCIImaze.Maze.maps.keys())
# The maze game loop is contained in the maze.solve() method. Call the constructor, and then enter a for loop
#  to receive outputs from maze.solve(), and respond with our inputs.
maze = ASCIImaze.Maze("maze_map3", show_moves=False, show_coords=False)
for output in maze.solve():
    if (not ready_for_input):
        # On the first iteration, we get the maze to solve. Bring it into the LLM, then goto next iteration; that
        #  will be the first opportunity to pass an input to the ASCIImaze solver.
        print("output:", output)
        # Going to pull the maze into the LLM's chat history now.
        user_prompt=f"""```{output}```"""
        print(user_prompt)
        maze_to_solve = get_completion(extract_maze_system_prompt, user_prompt)
        print('maze_single_line',maze_to_solve.content)
        ready_for_input = True
        continue

    if (need_context_update):
        # Grab context by passing '3' to ASCIIMaze. Isolated so can use stored context instead.
        # Must go to next iteration b/c only then will 'output' be updated with the 3x3 neighborhood.
        maze.get_user_move('3')
        need_context_update = False
        continue

    if "Maze solved!" in output:
        # Victory!
        print(output)
        break


    # Pull in the current 3x3 neighborhood, IF the current node does not already have a state. Otherwise, this is a
    #  backtracking situation.

    # Debug
    print(f"Output reads: {output}")
    if curr_node.name == "":
        # Case for a node not previously visited.
        user_prompt = f"```{output}```"
        neigh = get_completion(generate_thought_context_system_prompt, user_prompt)
        curr_node.name = neigh.content

        # print("Neighborhood:")
        # print("\n".join(curr_node.name.split("|")))


    # Evaluate the current position; this is actually evaluating the through generated in the previous round.
    # Possible results are {{valid moves}, <just the backtrack direction>}
    # Backtrack if so indicated, generate a new thought if there are other valid moves.
    # The ASCIImaze.maze.solve() method, the main game loop, outputs a message when you correctly solve the maze, you
    #  don't need to figure out if the LLM has won the game. Just check for the victory message. Done above.



    valid_moves_cleaned = evaluate_thought(curr_node.name, curr_node)
    print(f"Valid moves are {valid_moves_cleaned}")



    if len(valid_moves_cleaned) == 1 and valid_moves_cleaned[0] == curr_node.backtrack:
        # The only move detected is the backtrack direction, so follow it, both in the maze and the tree.
        maze.get_user_move(curr_node.backtrack)
        curr_node = tree_backtrack(curr_node, curr_node.backtrack)

        # Do not update context, the state in curr_node has already been updated. BUT this blows up an assumption :(
        need_context_update = False

        # The valid moves from the last eval step are no longer correct. Go to the next iteration, get a new move set.
        continue

    # Generate a thought
    # The LLM chooses a next move, which will be a child of the current node
    # Again, inner monologues are used to recreate the 2D mazes.
    user_prompt = f"""```{maze_to_solve.content}, {curr_node.name}```"""

    # Adding the backtrack move to thought generation. Remember to deal with root node (no parent).
    valid_moves_temp = ", ".join(valid_moves_cleaned)

    # Initial value for LLM_move, for loop entry
    LLM_move_cleaned = ['Z']

    # Loop to check for valid move.
    # LLM_move_cleaned should be a single-element list. If that element is not part of the valid move set, try again.
    while valid_moves_temp.find(LLM_move_cleaned[0]) < 0:

        #print(f"Moves going to LLM are {valid_moves_temp}")

        if curr_node.parent != None:
            LLM_move = get_completion(generate_thought_system_prompt.format(vmove=valid_moves_temp,
                                                                    backmove=curr_node.backtrack), user_prompt)
        else:
            LLM_move = get_completion(generate_thought_system_prompt.format(vmove=valid_moves_temp,
                                                      ackmove=None), user_prompt)
        LLM_move_temp = re.sub('\&\&\&.+\&\&\&', '', LLM_move.content, flags=re.DOTALL)
        # Again, another regex needed.
        LLM_move_cleaned = re.findall("[A-Z]", LLM_move_temp)
        print(f"I choose move {LLM_move_cleaned[0]}")
    # Error check
    if len(LLM_move_cleaned) > 1:
        print("Extra stuff in the generated thought, aborting")
        break


    # Make the chosen move, and add the corresponding node to the tree.
    maze.get_user_move(LLM_move_cleaned[0])
    curr_node = add_node(curr_node, LLM_move_cleaned[0])
    if curr_node is None:
        print("You goofed.")
        print(LLM_move_cleaned)
    # Debug
    counter = counter + 1
    #print(f"I choose move {LLM_move_cleaned}")
    print(f"This is move {counter}")
    print(" ")

    # Need the next 3x3 neighborhood.
    need_context_update = True

print("I finished")
print(f"\nTotal input (prompt) tokens used: {total_input_tokens}")
print(f"Total output (completion) tokens used: {total_output_tokens}")
print(f"Total tokens processed: {total_input_tokens + total_output_tokens}")





<>:240: SyntaxWarning: invalid escape sequence '\&'
<>:240: SyntaxWarning: invalid escape sequence '\&'
/tmp/ipython-input-2598231178.py:240: SyntaxWarning: invalid escape sequence '\&'
  LLM_move_temp = re.sub('\&\&\&.+\&\&\&', '', LLM_move.content, flags=re.DOTALL)


dict_keys(['maze_map1', 'maze_map2', 'maze_map3'])
output:  You have your starting position 'S', the end position 'E' and your current position will be indicated after every move with '*'.
    Walls are labeled as '#' and are impenetrable. This is done one turn at a time giving me the direction you would like to go
    (up 'U', down 'D', left 'L', right 'R'). You can also request a 3x3 grid of the immediate area around you with '3'.
#######
#S    #
# # # #
# # #E#
#######
Enter a move direction (U, D, L, R, or 3 (for a 3x3 of the current position)): 
``` You have your starting position 'S', the end position 'E' and your current position will be indicated after every move with '*'.
    Walls are labeled as '#' and are impenetrable. This is done one turn at a time giving me the direction you would like to go
    (up 'U', down 'D', left 'L', right 'R'). You can also request a 3x3 grid of the immediate area around you with '3'.
#######
#S    #
# # # #
# # #E#
#######
Enter a move direction

# Report

## Introduction

In this lab we used prompt engineering strategies to accompany a Tree-of-Thought prompting strategy to guide an LLM (GPT-4o-mini) to solve an ASCII-based maze game. The goal was to design a series of prompts that:


*   Flatten an NxM maze into a 1D string where rows were delimited with '|'.
*   Similarly flatten a 3x3 grid representing the immediate surroundings of the current state.
*   Choose a list from a list of valid moves.

To do this, minimal code changes were made, but the three prompts had to be designed to guide the LLM's decision making. A Troo-of-Thought strategy was employed for this assignment.

## Tree-of-Thought Prompting

Tree-of-Thought prompting is a form of reasoning that allowsa model to consider multiple options before making a decision. Instead of producing a single linear reasoning chain, Tree-of-Thougt builds a search tree where each node is a partial solution. The model evaluates branches, expands promising paths much like a human exploring possible strategies.
In this lab, this was implemented using a sort of DFS that explored one move sequence to completion before backtracking.

Based on this framework, the lab was conducted by making minimal code changes and prompting teh LLM to conduct various aspects of the maze solving process.

## Results

Results and tokens used for each maze are as follows:  

Maze 1:
*   Result: Success
*   Moves required: 15
*   Input tokens used: 5630
*   Output tokens used: 154
*   Total tokens used: 5784

Maze 2:
*   Result: Success
*   Moves required: 10
*   Input tokens used: 3893
*   Output tokens used: 118
*   Total tokens used: 4011

Maze 3:
*   Result: Success
*   Moves required: 5
*   Input tokens used: 2327
*   Output tokens used: 72
*   Total tokens used: 2399



As you can see, the prompts were sufficient for the LLM to make decisions that lead to the solved maze. On top of that, the maze solutions that were achieve through the techniques mentioned above were actual all the most optimal solution to the mazes, although this behavior isn't guaranteed for all mazes. Also, this prompting strategy, and its associated prompts could reasonably solve any number of properly formatted ASCII mazes, which is the ultimate goal of any solution to a general problem.

It might appear that that, given these results, this method of LLM prompting is a great method for maze solving purposes, but this was largely found to be untrue. Though the strategy was successful in achieving its maze solving goal, it's latency, probabilistic nature, and associated costs make it a problematic candidate for problems of this type. Communicating with a remote LLM, and the time it takes to generate a response makes this process very slow. Also, since these models aren't deterministic, the results aren't always consistent accross multiple runs, even on the same maze. Lastly, there are economic considerations that can't be ignore, and the fact that a solution costs money through it's access to the OpenAI API makes the long term and large scale growth of this kind of solution very unsustainable. Also, to achieve the corect result via prompting required significant effort through trial and error.

Therefore, I would not recommend this method to solve this problem. For concrete grid mazes, prefer a conventional algorithm. Tree of Thought prompting is useful as a teaching and research tool to stabilize multi-step LLM reasoning, but it's not the right engine for the task.

## Costs

As shown above, the costs per solution is seen to be between 2327-5630 tokens, depending on the maze. I wasn't able to find an exact figure for the price of tokens for gpt4o-mini, but allow me to illustrate the approximate (and likely upper bound) of the price per solution through a comparable gpt5-mini. The prices for tokens are $0.250 per 1 million input and $2 per 1 million output tokens. This translates to the following cost when factoring in tokens used and the split between input and output:



*   Maze 1: $0.00172

*   Maze 2: $0.001212

*   Maze 3: $0.000726

These costs are small, but could stack overtime, hence the verdict to avoid using LLMs to solve this sort of problem.


##LLMs and Spatial Reasoning

LLMs are decent at local spatial reasoning when you lock things down: serialize the map (|-delimited), list valid actions {U,D,L,R}, forbid walls/repeats, and they’ll usually pick a sensible next step. Tree-of-Thought + DFS cuts down thrashing and keeps progress moving, but the model is still stochastic and can misread the grid, so optimal and repeatable paths aren’t guaranteed. Bottom line is to use the LLM to propose candidates under strict formatting and let a classical solver (BFS/A*) handle the actual pathfinding for speed, determinism, and cost.

Modern research tends to agree with this standpoint; External benchmarks consistently show that LLMs/VLMs are shaky at true spatial reasoning without heavy scaffolding. In fact, a study analyzed 13 different models in 2025, and determined that average accuracy ≈ random on core spatial tasks, such as the maze solving task of this lab. Clearly, LLM, as the name suggests, is better for natural language applications, and is far weaker in the area of spatial resoning. These findings line up with our lab experience, LLMs can follow local rules when tightly constrained, but global layout and long-horizon planning remain brittle.

## Conclusion

With Tree-of-Thought and strict prompts, GPT-4o-mini solved all three mazes, but it needed tight scaffolding and still behaved stochastically. Each run cost about $0.0007–$0.0017 at our rates and added latency. For grid mazes, classical BFS/DFS/A* is faster, deterministic, and basically free. Keep the LLM for parsing state, proposing candidates, or explaining a route and probably don’t use it as the pathfinder.






## Sources


*   gpt4o-mini pricing: https://openai.com/api/pricing/
*   spatial reasoning study: https://arxiv.org/html/2503.19707v1?utm_source=chatgpt.com
*   ECE 449 course material
*   ECE 449 Lab 2 Document







